In [0]:
%sql
-- Databricks SQL script to create views in 5_projects.clix for data migration
-- These views will prepare data for upload to CorePlatform tables
-- Based on the cohort: MRNs with echocardiograms and specific document types

USE CATALOG 5_projects;
USE SCHEMA clix;

-- =============================================
-- BASE VIEW: Define document selection criteria ONCE
-- This view contains all the logic for identifying relevant documents
-- =============================================
CREATE OR REPLACE VIEW v_clix_base_documents AS
WITH document_categorization AS (
    SELECT
        bd.*,
        -- Categorize documents
        CASE 
            WHEN LOWER(bd.ChildEvent) LIKE '%us transthoracic echocardiogram%'
                OR LOWER(bd.ChildTagText) LIKE '%us transthoracic echocardiogram%'
                OR LOWER(bd.MainTitleText) LIKE '%us transthoracic echocardiogram%'
                THEN 'Echocardiogram'
            
            WHEN LOWER(bd.MainEventDesc) LIKE '%clerking%' 
                OR LOWER(bd.MainTitleText) LIKE '%clerking%' 
                OR LOWER(bd.ChildEvent) LIKE '%clerking%' 
                OR LOWER(bd.EventDesc) LIKE '%clerking%' 
                THEN 'Clerking'
            
            WHEN (LOWER(bd.MainEventDesc) LIKE '%cardiology%' AND LOWER(bd.MainEventDesc) LIKE '%mdt%')
                OR (LOWER(bd.MainTitleText) LIKE '%cardiology%' AND LOWER(bd.MainTitleText) LIKE '%mdt%')
                OR (LOWER(bd.ChildEvent) LIKE '%cardiology%' AND LOWER(bd.ChildEvent) LIKE '%mdt%')
                OR (LOWER(bd.EventDesc) LIKE '%cardiology%' AND LOWER(bd.EventDesc) LIKE '%mdt%')
                THEN 'Cardiology MDT letters'
            
            WHEN (LOWER(bd.MainEventDesc) LIKE '%cardiothoracic%' AND LOWER(bd.MainEventDesc) LIKE '%mdt%')
                OR (LOWER(bd.MainTitleText) LIKE '%cardiothoracic%' AND LOWER(bd.MainTitleText) LIKE '%mdt%')
                OR (LOWER(bd.ChildEvent) LIKE '%cardiothoracic%' AND LOWER(bd.ChildEvent) LIKE '%mdt%')
                OR (LOWER(bd.EventDesc) LIKE '%cardiothoracic%' AND LOWER(bd.EventDesc) LIKE '%mdt%')
                THEN 'Cardiothoracic MDT letters'
            
            WHEN LOWER(bd.MainEventDesc) LIKE '%cardiology%letter%' 
                OR LOWER(bd.MainTitleText) LIKE '%cardiology%letter%' 
                OR LOWER(bd.ChildEvent) LIKE '%cardiology%letter%' 
                OR LOWER(bd.EventDesc) LIKE '%cardiology%letter%' 
                OR (LOWER(bd.MainEventDesc) LIKE '%cardiology%' AND LOWER(bd.MainEventDesc) LIKE '%clinic%')
                THEN 'Cardiology letters'
            
            WHEN LOWER(bd.MainEventDesc) LIKE '%cardiothoracic%letter%' 
                OR LOWER(bd.MainTitleText) LIKE '%cardiothoracic%letter%' 
                OR LOWER(bd.ChildEvent) LIKE '%cardiothoracic%letter%' 
                OR LOWER(bd.EventDesc) LIKE '%cardiothoracic%letter%' 
                OR (LOWER(bd.MainEventDesc) LIKE '%cardiothoracic%' AND LOWER(bd.MainEventDesc) LIKE '%clinic%')
                THEN 'Cardiothoracic letters'
            
            WHEN LOWER(bd.MainEventDesc) LIKE '%discharge%summar%' 
                OR LOWER(bd.MainTitleText) LIKE '%discharge%summar%' 
                OR LOWER(bd.ChildEvent) LIKE '%discharge%summar%' 
                OR LOWER(bd.EventDesc) LIKE '%discharge%summar%' 
                THEN 'Discharge summaries'
            
            ELSE 'Other'
        END AS document_category,
        -- For echocardiograms, rank to get first one per MRN
        ROW_NUMBER() OVER (
            PARTITION BY bd.MRN, 
            CASE 
                WHEN LOWER(bd.ChildEvent) LIKE '%us transthoracic echocardiogram%'
                    OR LOWER(bd.ChildTagText) LIKE '%us transthoracic echocardiogram%'
                    OR LOWER(bd.MainTitleText) LIKE '%us transthoracic echocardiogram%'
                    THEN 1 
                ELSE 0 
            END
            ORDER BY bd.ClinicalSignificantDate, bd.EventID
        ) AS echo_rank
    FROM 5_projects.clix.mrn_list m
    INNER JOIN 4_prod.rde.rde_blobdataset bd
        ON m.MRN = bd.MRN
)
SELECT 
    *,
    CASE 
        WHEN document_category = 'Echocardiogram' AND echo_rank = 1 
        THEN 1 
        ELSE 0 
    END AS is_first_echo
FROM document_categorization
WHERE 
    -- Include only first echocardiogram
    (document_category = 'Echocardiogram' AND echo_rank = 1)
    -- Include all other relevant document types
    OR document_category IN (
        'Clerking',
        'Cardiology letters',
        'Cardiothoracic letters',
        'Discharge summaries',
        'Cardiology MDT letters',
        'Cardiothoracic MDT letters'
    );

-- =============================================
-- 1. PATIENT VIEW
-- Uses PERSON_ID as primary identifier
-- Simply joins to base documents to get relevant patients
-- =============================================
CREATE OR REPLACE VIEW v_clix_patients AS
SELECT DISTINCT
    pd.PERSON_ID as person_id,
    pd.MRN as mrn,  -- Will go to PatientAliases.Identifier
    pd.NHS_Number as nhs_number,  -- Additional alias
    CAST(NULL AS STRING) as forename,  -- Not available in source
    CAST(NULL AS STRING) as surname,   -- Not available in source
    pd.Date_of_Birth as date_of_birth,  -- NOT NULL in destination
    pd.Gender as gender_code,  -- Will need lookup to GenderId
    pd.Ethnicity as ethnicity_code,  -- Will need lookup to EthnicityId
    pd.Postcode as zip,  -- Maps to Patients.Zip
    CASE WHEN pd.Date_of_Death IS NOT NULL THEN 1 ELSE 0 END as deceased_status,  -- NOT NULL bit
    pd.Date_of_Death as date_of_death,
    'Barts' as namespace  -- For PatientAliases
FROM 4_prod.rde.rde_patient_demographics pd
WHERE EXISTS (
    SELECT 1 
    FROM v_clix_base_documents bd
    WHERE bd.PERSON_ID = pd.PERSON_ID
);

-- =============================================
-- 2. ENCOUNTER VIEW
-- Links actual encounters to relevant documents via mill_clinical_event
-- =============================================
CREATE OR REPLACE VIEW v_clix_encounters AS
SELECT DISTINCT
    e.PERSON_ID,
    e.ENCNTR_ID,
    CAST(e.ENCNTR_ID AS STRING) as encounter_id,  -- Will go to EncounterAliases.Identifier
    e.MRN,
    e.CREATE_DT_TM as admission_datetime,
    CAST(NULL AS TIMESTAMP) as discharge_datetime,  -- Not available in rde_encounter
    e.ENC_TYPE as visit_class_code,  -- Will need lookup to VisitClassId
    e.ADMIN_DESC as visit_type_code,  -- Will need lookup to VisitTypeId  
    CAST(NULL AS STRING) as location_code,  -- Not available, LocationId is nullable
    1 as facility_id_default,  -- FacilityId is NOT NULL, will need actual value
    'Barts' as namespace  -- For EncounterAliases
FROM 4_prod.rde.rde_encounter e
WHERE EXISTS (
    -- Link through clinical_event table to our specific events
    SELECT 1
    FROM v_clix_base_documents bd
    INNER JOIN 4_prod.raw.mill_clinical_event ce
        ON ce.EVENT_ID = bd.EventID 
        AND ce.PERSON_ID = bd.PERSON_ID
    WHERE ce.ENCNTR_ID = e.ENCNTR_ID
    AND ce.PERSON_ID = e.PERSON_ID
);

-- =============================================
-- 3. DOCUMENT VIEW
-- Links documents to their associated encounters
-- =============================================
CREATE OR REPLACE VIEW v_clix_documents AS
SELECT
    CAST(bd.EventID AS STRING) as source_id,  -- Documents.SourceID
    'Barts' as source_id_namespace,  -- Documents.SourceIDNamespace
    bd.PERSON_ID,
    bd.MRN,
    ce.ENCNTR_ID,
    COALESCE(bd.MainEventDesc, bd.MainTitleText, bd.EventDesc, 'Clinical Document') as title,  -- NOT NULL
    CAST(bd.ClinicalSignificantDate AS TIMESTAMP) as observation_datetime,  -- NOT NULL
    bd.BlobContents as content,  -- NOT NULL
    bd.EventID as event_id,
    bd.document_category,
    CASE 
        WHEN bd.is_first_echo = 1 
        THEN 'First Echocardiogram'
        ELSE bd.document_category
    END as document_type_detail
FROM v_clix_base_documents bd
LEFT JOIN 4_prod.raw.mill_clinical_event ce
    ON ce.EVENT_ID = bd.EventID
    AND ce.PERSON_ID = bd.PERSON_ID
WHERE bd.BlobContents IS NOT NULL  -- Content is NOT NULL in destination
AND bd.ClinicalSignificantDate IS NOT NULL;  -- ObservationDateTime is NOT NULL

-- =============================================
-- SUMMARY VIEW (Optional)
-- Shows document counts by type for validation
-- =============================================
CREATE OR REPLACE VIEW v_clix_document_summary AS
SELECT 
    MRN,
    document_type_detail,
    COUNT(*) as document_count
FROM v_clix_documents
GROUP BY MRN, document_type_detail
ORDER BY MRN, document_type_detail;